In [1]:
# підключення потрібних модулей
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import shutil
from pprint import pprint

# параметри виведення
pd.set_option("display.max_columns", 500) # кількість колонок
pd.set_option("display.max_rows", 1000) # кількість рядків
pd.set_option("display.max_colwidth", 300) # ширина колонок
pd.set_option("display.precision", 5) # кількість знаків після коми

# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# магічний метод для того щоб отримувати графіки біля комірок з кодом
%matplotlib inline

In [2]:
# додаткові модулі
from BTrees.OOBTree import OOBTree
import csv
from copy import deepcopy
from pprint import pprint
import timeit

In [3]:
# завантаження даних
with open('generated_items_data.csv') as f:
    reader = list(csv.DictReader(f))

# огляд перших трьох рядків
for row in reader[:3]:
    print(row)

{'ID': '88519', 'Name': 'Product_88519', 'Category': 'Home', 'Price': '375.01'}
{'ID': '73117', 'Name': 'Product_73117', 'Category': 'Toys', 'Price': '107.01'}
{'ID': '68277', 'Name': 'Product_68277', 'Category': 'Toys', 'Price': '131.46'}


In [4]:
# завантаження даних в словник
def add_item_to_dict(csv_row):
    key = int(csv_row.pop("ID"))
    obj = csv_row
    data_dict[key] = obj

data_dict = {}
for row in deepcopy(reader):
    add_item_to_dict(row)

len(data_dict)

100000

In [5]:
# завантаження даних в OOBTree
def add_item_to_tree(csv_row):
    key = int(csv_row.pop("ID"))
    obj = csv_row
    tree.update({key: obj})

tree = OOBTree()
for row in deepcopy(reader):
    add_item_to_tree(row)

tree

In [6]:
# функція пошуку в словнику діапазону елементів за ключем
def range_query_dict(start, end):
    """
    Функція для виконання запиту діапазону в словнику
    :param start: початок діапазону
    :param end: кінець діапазону
    :return: список елементів у діапазоні
    """
    result = []
    for key in data_dict.keys():
        if start <= key <= end:
            result.append({key: data_dict[key]})
    return result

# тестування функції
print(f"Range query in dict {range_query_dict(1000, 1100)}")

Range query in dict [{1063: {'Name': 'Product_1063', 'Category': 'Sports', 'Price': '104.43'}}, {1026: {'Name': 'Product_1026', 'Category': 'Home', 'Price': '168.85'}}, {1045: {'Name': 'Product_1045', 'Category': 'Clothing', 'Price': '157.53'}}, {1039: {'Name': 'Product_1039', 'Category': 'Sports', 'Price': '86.09'}}, {1083: {'Name': 'Product_1083', 'Category': 'Books', 'Price': '165.82'}}, {1081: {'Name': 'Product_1081', 'Category': 'Clothing', 'Price': '113.61'}}, {1070: {'Name': 'Product_1070', 'Category': 'Toys', 'Price': '145.94'}}, {1080: {'Name': 'Product_1080', 'Category': 'Home', 'Price': '335.25'}}, {1054: {'Name': 'Product_1054', 'Category': 'Home', 'Price': '230.37'}}, {1031: {'Name': 'Product_1031', 'Category': 'Home', 'Price': '489.97'}}, {1033: {'Name': 'Product_1033', 'Category': 'Sports', 'Price': '436.18'}}, {1053: {'Name': 'Product_1053', 'Category': 'Clothing', 'Price': '157.56'}}, {1096: {'Name': 'Product_1096', 'Category': 'Clothing', 'Price': '209.52'}}, {1030: {

In [7]:
# функція пошуку в дереві діапазону елементів за ключем
def range_query_tree(start, end):
    """
    Функція для виконання запиту діапазону в OOBTree
    :param start: початок діапазону
    :param end: кінець діапазону
    :return: список елементів у діапазоні
    """
    return list(tree.items(start, end))

# тестування функції
print(f"Range query in tree {range_query_tree(1000, 1100)}")

Range query in tree [(1000, {'Name': 'Product_1000', 'Category': 'Sports', 'Price': '247.95'}), (1001, {'Name': 'Product_1001', 'Category': 'Toys', 'Price': '336.59'}), (1002, {'Name': 'Product_1002', 'Category': 'Electronics', 'Price': '268.81'}), (1003, {'Name': 'Product_1003', 'Category': 'Toys', 'Price': '216.03'}), (1004, {'Name': 'Product_1004', 'Category': 'Books', 'Price': '364.03'}), (1005, {'Name': 'Product_1005', 'Category': 'Books', 'Price': '183.6'}), (1006, {'Name': 'Product_1006', 'Category': 'Home', 'Price': '26.53'}), (1007, {'Name': 'Product_1007', 'Category': 'Home', 'Price': '98.13'}), (1008, {'Name': 'Product_1008', 'Category': 'Books', 'Price': '145.01'}), (1009, {'Name': 'Product_1009', 'Category': 'Home', 'Price': '252.62'}), (1010, {'Name': 'Product_1010', 'Category': 'Home', 'Price': '140.58'}), (1011, {'Name': 'Product_1011', 'Category': 'Electronics', 'Price': '340.52'}), (1012, {'Name': 'Product_1012', 'Category': 'Electronics', 'Price': '65.2'}), (1013, {'

In [8]:
# підготовка рядків коду для замірів часу виконання

dict_code_text = """
def range_query_dict(start, end):
    result = []
    for key in data_dict.keys():
        if start <= key <= end:
            result.append({key: data_dict[key]})
    return result

range_query_dict(1000, 10000)
    """

tree_code_text = """
def range_query_tree(start, end):
    return list(tree.items(start, end))

range_query_tree(1000, 10000)
"""

In [13]:
# # замір часу виконання функцій

n = 100

dict_timer = timeit.timeit(
    stmt=dict_code_text,
    setup="from __main__ import range_query_dict, data_dict",
    number=n,
)

tree_timer = timeit.timeit(
    stmt=tree_code_text,
    setup="from __main__ import range_query_dict, tree",
    number=n,
)

# виведення результатів
print(f'Number of iterations: {n}')
print(f"Total range_query time for OOBTree: {tree_timer:.5f} seconds")
print(f"Total range_query time for Dict: {dict_timer:.5f} seconds")


Number of iterations: 100
Total range_query time for OOBTree: 0.06356 seconds
Total range_query time for Dict: 0.64028 seconds


Висновки:
---------
1. Пошук ключів словника в діапазоні відбувається ориентовно в 10 раз повільніше ніж в OOBTree тому, що в словнику йде лінійний перебір усіх значень незалежно від розміру шуканого діапазону. В дереві скоріше за все йде прямий відбор ключів від початкового до кінечного значення, що значно швидше, ніж перебір всіх елементів словника. 